
## Participant sheets cleanup procedure

#### Invalid particinant sheets have ascii protected headers with a length exceeding 32767 bytes



In [9]:
pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-1.2.0-cp38-cp38-manylinux1_x86_64.whl (9.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.4
    Uninstalling pandas-1.1.4:
      Successfully uninstalled pandas-1.1.4
Note: you may need to restart the kernel to use updated packages.


### To Remember

 - 78 images & 39 control (gray square) stimuli per session
 - Thus, nTrials = nStim x 1.5 = 
 - 103 valid participants


In [1]:
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from chardet import detect
from chardet.universaldetector import UniversalDetector as udet
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from tqdm import tqdm
from cimaq_utils import flatten
from cimaq_utils import loadimages
from cimaq_utils import get_encoding

# set max display size options for pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_colwidth', None)

In [2]:
indir = xpu('~/cimaq_memory/extracted_eprime_cimaq_03-19')
p1 = pd.read_csv(join(indir, "participants.tsv"),
                     sep='\t').set_index("sub-ID")
p1["sheetpaths"] = [[join(indir, row[0], sheet)
                     for sheet in ls(join(indir, row[0]))
                     if os.path.isfile(join(indir, row[0], sheet))
                     and sheet.startswith(".") == False]
                    for row in p1.iterrows()]
prefixes = sorted(list(dict.fromkeys([bname(item).split("CIMAQ", 1)[0] 
                    for item in flatten(p1.sheetpaths)
                    if "CIMAQ" in bname(item)])))
p2 = df([[[item for item in row[1]["sheetpaths"]
           if pref in item][0] for pref in prefixes]
        for row in p1.iterrows()],
       index=p1.index,
       columns=prefixes).sort_index()

results = df(([get_encoding(sheet)[1]["encoding"]
               for sheet in row[1]]
               for row in p2.iterrows()),
            index=p2.index, columns=p2.columns)
test = df((tuple(zip(p2.loc[ind].values,
                     results.loc[ind].values))
           for ind in p2.index),
            index=p2.index, columns=p2.columns)
display(test)

,Onset-Event-Encoding_,Output-Responses-Encoding_,Output_Retrieval_
sub-ID,,,
sub-3025432,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3025432/Onset-Event-Encoding_CIMAQ_3025432_session1B, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3025432/Output-Responses-Encoding_CIMAQ_3025432_session1B, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3025432/Output_Retrieval_CIMAQ_3025432_1, ISO-8859-1)"
sub-3123186,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3123186/Onset-Event-Encoding_CIMAQ_3123186_session1A, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3123186/Output-Responses-Encoding_CIMAQ_3123186_session1A, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3123186/Output_Retrieval_CIMAQ_3123186_1, ISO-8859-1)"
sub-3163875,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3163875/Onset-Event-Encoding_CIMAQ_3163875_session1C, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3163875/Output-Responses-Encoding_CIMAQ_3163875_session1C, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3163875/Output_Retrieval_CIMAQ_3163875_1, ISO-8859-1)"
sub-3291977,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3291977/Onset-Event-Encoding_CIMAQ_3291977_session2 D, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3291977/Output-Responses-Encoding_CIMAQ_3291977_session2 D, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3291977/Output_Retrieval_CIMAQ_3291977_1, ISO-8859-1)"
sub-3303819,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3303819/Onset-Event-Encoding_CIMAQ_3303819, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3303819/Output-Responses-Encoding_CIMAQ_3303819, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3303819/Output_Retrieval_CIMAQ_3303819, ISO-8859-1)"
sub-3400972,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3400972/Onset-Event-Encoding_CIMAQ_3400972_session1D, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3400972/Output-Responses-Encoding_CIMAQ_3400972_session1D, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3400972/Output_Retrieval_CIMAQ_3400972_1, ISO-8859-1)"
sub-3420680,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3420680/Onset-Event-Encoding_CIMAQ_3420680_session1B, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3420680/Output-Responses-Encoding_CIMAQ_3420680_session1B, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3420680/Output_Retrieval_CIMAQ_3420680_1, ISO-8859-1)"
sub-3431566,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3431566/Onset-Event-Encoding_CIMAQ_3431566_session1C, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3431566/Output-Responses-Encoding_CIMAQ_3431566_session1C, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3431566/Output_Retrieval_CIMAQ_3431566_1, ISO-8859-1)"
sub-3494204,"(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3494204/Onset-Event-Encoding_CIMAQ_3494204_session1D, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3494204/Output-Responses-Encoding_CIMAQ_3494204_session1D, ascii)","(/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3494204/Output_Retrieval_CIMAQ_3494204_1, ISO-8859-1)"


#### Previously displayed DataFrame facilitates participant-wise identification of files 
#### Here, each participant is indexed in allsheets as row[0] (index) according to
#### participant ID & sheet name

In [3]:
def getid(sheet):
    nameparts = [str(item) for item in os.path.splitext(bname(sheet))[0].split("_")]
    subid = "sub-"+str(nameparts[nameparts.index(\
                 (item for item in nameparts
                   if item.isnumeric()).__next__())])
    return subid

def makename(sheet):
    if prefixes[0] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-memory_events_onset"+".tsv")
    if prefixes[1] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-memory_events_output"+".tsv")
    if prefixes[2] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-Retrieval"+".tsv")
    else:
        print("python no compute: "+sheet)
# def makename_ret(sheet):
#     return 
tname = makename("/home/fnadeau/extracted_eprime2020/sub-9929164/Output_Retrieval_CIMAQ_9929164_1.txt")
print(tname)

/home/fnadeau/extracted_eprime2020/sub-9929164/sub-9929164_task-Retrieval.tsv


In [4]:
def get_enc_onsets(frame):

    # Names from M. St-Laurent (2019)
    EncOnsetCols = ["TrialNum", "Condition", "TrialNum_perCondi",
                    "ImageID", "Trial_part", "onsetSec", "durationSec"]
    sheets = frame.iloc[:,0]
    s_ids, yallofems = [], []
    for sheet in sheets:
        newsheet = pd.read_fwf(sheet[0], encoding=sheet[1],
                               header=None, sep='\t',
                               names=EncOnsetCols).fillna(False).iloc[6:]

        stimids = newsheet[["ImageID",
                           "TrialNum_perCondi"]].drop_duplicates(
                      subset=["ImageID",
                              "TrialNum_perCondi"]).reset_index(drop=True)
        s_ids.append((bname(sheet[0]), stimids))
        newsheet.to_excel(join(indir, "temp_"+bname(sheet[0])+'.xlsx'))
        newsheet = pd.read_excel(join(
            indir, "temp_"+bname(sheet[0])+'.xlsx'),
                                engine='openpyxl').drop(
                ['TrialNum_perCondi', 'Condition'], axis=1)

        tempsheet = newsheet[['TrialNum', 'Trial_part',
                              'onsetSec', 'durationSec']]

        # Extract and concatenate relevant info
        fixsheet = df([sheet[1] for sheet in tempsheet.iterrows()
                       if sheet[1]['Trial_part'] == 'Fixation'])
        timing = tempsheet.loc[[sheet[0]
                                for sheet in tempsheet.iterrows()
                                if sheet[0] not in fixsheet.index]]
        fixsheet = fixsheet.rename(columns={"onsetSec": "fixOnsetSec",
                                            "durationSec": "fixDurSec"})
        fixsheet = fixsheet.transpose().iloc[-2:].transpose(\
                       ).reset_index(drop=True)
        timing = timing.rename(columns={"onsetSec": "stimOnsetSec",
                                        "durationSec": "stimDurSec"})
        timing = timing.transpose().iloc[-2:].transpose().reset_index(drop=True)
        allofem = pd.concat([timing, fixsheet, stimids], axis=1, sort=False)
        allofem = allofem.rename(columns={"TrialNum_perCondi": "tNumXcond"})
#         allofem = allofem.set_index("Unnamed: 0").reset_index(drop=True)
        enconsTuple = (makename(sheet[0]), allofem)
        yallofems.append(enconsTuple)
#         enconsTuple[1].to_csv(enconsTuple[0], sep='\t')

#         yallofems.append(enconsTuple)
#         os.remove(join(indir, "temp_"+bname(sheet[0])+'.xlsx'))
    return sorted(yallofems)

#     test["newEncOnsetPaths"] = [os.path.splitext(item[0])[0]+".tsv"
#                                 for item in yallofems]

enconsets = get_enc_onsets(test)


In [5]:
display(enconsets)

[('/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3025432/sub-3025432_task-memory_events_onset.tsv',
      stimOnsetSec stimDurSec fixOnsetSec  \
  0         11.542        3.0      14.551   
  1         20.051        3.0      23.060   
  2         23.561        3.0      26.570   
  3         27.557        3.0      30.566   
  4         31.569        3.0      34.578   
  5         36.065        3.0      39.074   
  6         41.064        3.0      44.073   
  7         47.065        3.0      50.074   
  8         50.575        3.0      53.584   
  9         57.579        3.0      60.588   
  10        63.079        3.0      66.088   
  11        67.576        3.0      70.585   
  12        72.574        3.0      75.583   
  13        79.077        3.0      82.086   
  14        82.587        3.0      85.596   
  15        88.087        3.0      91.096   
  16        91.597        3.0      94.606   
  17        95.593        3.0      98.602   
  18       100.089        3.0 

In [6]:
def get_enc_outputs(frame):
    sheets = frame.iloc[:,1]
    encsheets = []
    for sheet in sheets:
        encsheet = pd.read_csv(sheet[0],
                               encoding=sheet[1],
                               header=0,
                               sep='\t').fillna(False).rename(
                       columns={"TrialNumber": "TrialNum",
                                "Category": "Condition"}).iloc[3:]
        encsheet = encsheet.set_index("TrialNum")
        # Drop duplicate columns with enc_onset's
        encsheet = encsheet.drop(["TrialCode", "OldNumber", "Stim_RESP"], axis=1)
        encsheet["Condition"] = encsheet["Condition"].astype(\
                                    'str')
        # Convert ms to s (as are all other time measures)
        encsheet["Stim_RT"] = [val/1000 for val in encsheet["Stim_RT"]]

        encsheet = encsheet.drop(["Stim_ACC"], axis=1)
        encsheet = encsheet.reset_index(drop=True)
        encoutTuple = (makename(sheet[0]), encsheet)
        encsheets.append(encoutTuple)
        [encoutTuple[1].to_csv(encoutTuple[0], sep='\t')
         for encoutTuple in encsheets]

    return sorted(encsheets)
encoutputs = get_enc_outputs(test)
# display([[item for item in ls(join(indir, row[0]))
#          if item.endswith(".tsv")]
#          for row in test.iterrows()])
# # display(allencoutputs[0][1].iloc[:,-1:], allencoutputs.tail(1))
# display(allencoutputs[0])

In [7]:
display(encoutputs)


[('/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3025432/sub-3025432_task-memory_events_output.tsv',
      Condition  CorrectSource  Stim_RT
  0         CTL              8    1.402
  1         Enc              6    0.000
  2         Enc              5    1.822
  3         Enc              6    1.930
  4         Enc              9    1.814
  5         Enc              8    1.094
  6         Enc              9    2.895
  7         CTL              5    0.838
  8         CTL              6    0.807
  9         Enc              8    2.139
  10        CTL              8    0.791
  11        Enc              6    1.678
  12        Enc              5    2.383
  13        CTL              8    0.840
  14        Enc              6    2.578
  15        Enc              6    1.558
  16        Enc              8    2.848
  17        Enc              5    2.340
  18        Enc              9    2.740
  19        Enc              9    2.095
  20        Enc              5    1.966
  21

In [8]:
def get_ret_outputs(frame):
    sheets = frame.iloc[:,2]
    retsheets = []
    for sheet in sheets:
        # Removing last column as an Eprime error occured (St-Laurent, 2019)
        retsheet = pd.read_csv(sheet[0],
                               encoding=sheet[1],
                               header=0,
                               sep='\t').fillna(False).iloc[:, :-1]
        retsheet["Recognition_RT"] = [float(val)/1000 for val in retsheet["Recognition_RT"]]
        retTuple = (makename(sheet[0]), retsheet)
        retsheets.append(retTuple)
#         retTuple[1].to_csv(retTuple[0], sep='\t')
    return sorted(retsheets)

retsheets = get_ret_outputs(test)
display(retsheets[0])

('/home/francois/cimaq_memory/extracted_eprime_cimaq_03-19/sub-3025432/sub-3025432_task-Retrieval.tsv',
     category                             Stim  \
 0        OLD               animal_penguin.bmp   
 1        OLD         sporting_bicycle_old.bmp   
 2        OLD              fruit_lemon_new.bmp   
 3        New               vegie_broccoli.bmp   
 4        OLD             animal_zebra_old.bmp   
 5        OLD                Kitchen_apron.bmp   
 6        OLD           animal_monarch_old.bmp   
 7        New          kitchen_toaster_old.bmp   
 8        New          musical_uculele_new.bmp   
 9        New                 fruit_cherry.bmp   
 10       New               food_walnut01c.bmp   
 11       OLD               food_pickle01a.bmp   
 12       New                vegie_pumpkin.bmp   
 13       OLD              animal_lion_new.bmp   
 14       New            sporting_flippers.bmp   
 15       OLD       sporting_boxing_gloves.bmp   
 16       New               animal_gorilla.bmp

In [9]:
sheetslist = df(([join(indir, row[0], sheet)
                  for sheet in ls(join(indir, row[0]))
                  if os.path.splitext(sheet)[1] == ".tsv"]
                 for row in test.iterrows()),
                index=test.index)
sheetspersub = [tuple(pd.read_csv(val, sep='\t') for val in row[1].values)
               for row in sheetslist.iterrows()]
fullencsheets = [(test.index[item[0]],
                  pd.concat([item[1][0], item[1][1]],
                            axis=1, sort=False).reset_index(drop=True).drop("Unnamed: 0", axis=1))
                 for item in enumerate(sheetspersub)]


[item[1].to_csv(join(indir, item[0], item[0]+"_task-Memory_events.tsv"), sep='\t')
 for item in fullencsheets]
test['newdirs'] = [xpu(join("~/cimaq_events2021", row[0]))
                             for row in test.iterrows()]
[os.makedirs(ndir, exist_ok=True) for ndir in test.newdirs]
oldevnts = flatten([[sheet for sheet in loadimages(join(indir, row[0]))
                     if "task-Memory_events.tsv" in sheet]
                    for row in test.iterrows()])
oldrets = flatten([[sheet for sheet in loadimages(join(indir, row[0]))
                     if "_task-retrieval" in sheet]
                    for row in test.iterrows()])
newevnts = flatten([[join(row[1]['newdirs'], bname(sheet))
                     for sheet in loadimages(join(indir, row[0]))
                     if "task-Memory_events.tsv" in sheet]
                     for row in test.iterrows()])
newrets = flatten([[join(row[1]['newdirs'], bname(sheet))
                     for sheet in loadimages(join(indir, row[0]))
                     if  "_task-retrieval" in sheet]
                     for row in test.iterrows()])
                    


# testsheets = [pd.concat(pd.[item[0], item[1]], axis=1, sort=False)
#               for item in sheetslist]


In [10]:
evtmvr = tuple(zip(oldevnts, newevnts))
retmvr = tuple(zip(oldrets, newrets))
# display(newrets)
[shutil.move(item[0], item[1]) for item in evtmvr]
[shutil.move(item[0], item[1]) for item in retmvr]

[]

In [11]:
savior = tuple(zip(test.sort_index().index, [item[1] for item in fullencsheets]))
# display(savior[0][0])
[item[1].to_csv(join(xpu('~/cimaq_events2021'), item[0], item[0]+"_task-Memory_events.tsv"), sep='\t')
 for item in savior]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [12]:
display(mover)

NameError: name 'mover' is not defined

In [679]:
def mkarchv(filepath):
    print 'creating archive'
    zf = zipfile.ZipFile(filepath, mode='w')
    try:
        print 'adding README.txt'
        zf.write(filepath)
    finally:
        print 'closing'
        zf.close()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('creating archive')? (<ipython-input-679-6258df959554>, line 2)

In [ ]:
def removeComments(string):
    string = re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,string) # remove all occurrences streamed comments (/*COMMENT */) from string
    string = re.sub(re.compile("//.*?\n" ) ,"" ,string) # remove all occurrence single-line comments (//COMMENT\n ) from string
    return string